<a href="https://colab.research.google.com/github/marioschuele/masterarbeit/blob/main/SOTA_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib

In [ ]:
! pip install kaggle

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


In [ ]:
%%capture
!kaggle datasets download yuweisunut/sidd-segmented-intrusion-detection-dataset
!unzip sidd-segmented-intrusion-detection-dataset

In [ ]:
benign_images = []
typeA_images = []
typeB_images = []

In [ ]:
import os
import pandas as pd
directory = 'SIDD'

In [ ]:
imgs = {}
uid = 0

for client in os.listdir(directory):
  curr_path = f'{directory}/{client}/pcap'

  for subdir in os.listdir(curr_path):
    curr_path = f'{directory}/{client}/pcap/{subdir}/dataset'
    curr_type = subdir[-1:]
    
    for dayscen in os.listdir(curr_path):
      curr_path = f'{directory}/{client}/pcap/{subdir}/dataset/{dayscen}'
      #+print(dayscen)
      for img in os.listdir(curr_path):
        if dayscen == 'benign':
            imgs[uid] = {'id': uid, 'type': str(0), 'fn': img, 'path': curr_path + '/' + img}
        elif dayscen == 'malicious':
            imgs[uid] = {'id': uid, 'type': str(curr_type), 'fn': img, 'path': curr_path + '/' + img}
        uid +=1


In [ ]:
imgs[1]

In [ ]:
img_df = pd.DataFrame.from_dict(imgs,orient='index')
img_df

,id,type,fn,path
0,0,0,623.jpg,SIDD/n005/pcap/n005_20191127_000001_1/dataset/...
1,1,0,99.jpg,SIDD/n005/pcap/n005_20191127_000001_1/dataset/...
2,2,0,252.jpg,SIDD/n005/pcap/n005_20191127_000001_1/dataset/...
3,3,0,661.jpg,SIDD/n005/pcap/n005_20191127_000001_1/dataset/...
4,4,0,278.jpg,SIDD/n005/pcap/n005_20191127_000001_1/dataset/...
...,...,...,...,...
1385579,1385579,3,350.jpg,SIDD/n053/pcap/n053_20191027_000001_3/dataset/...
1385580,1385580,3,251.jpg,SIDD/n053/pcap/n053_20191027_000001_3/dataset/...
1385581,1385581,3,384.jpg,SIDD/n053/pcap/n053_20191027_000001_3/dataset/...
1385582,1385582,3,352.jpg,SIDD/n053/pcap/n053_20191027_000001_3/dataset/...


In [ ]:
nodes = ['n005', 'n006', 'n008', 'n031', 'n034', 'n035', 'n036', 'n038', 'n041', 'n045', 'n046', 'n047', 'n048', 'n053', 'n056']
classes = [0,1,3]#['Benign', 'Server Message Block', 'TCP SYN flood attack']

image_size = (48,48)
batch_size = 50

train_datagen = ImageDataGenerator(rescale=10, validation_split=0.2)
train_batches = train_datagen.flow_from_dataframe(
    img_df,
    x_col="path",
    y_col="type",
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'categorical',
    subset = 'training'
)

validation_batches = train_datagen.flow_from_dataframe(
    img_df,
    x_col="path",
    y_col="type",
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'categorical',
    subset = 'validation'
)

test_batches = train_datagen.flow_from_dataframe(
    img_df,
    x_col="path",
    y_col="type",
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle = False,
    subset = 'validation'
)



Found 1108468 validated image filenames belonging to 3 classes.
Found 277116 validated image filenames belonging to 3 classes.
Found 277116 validated image filenames belonging to 3 classes.


In [14]:
model = tf.keras.applications.EfficientNetV2M(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=2,
)

model.compile(
    optimizer=keras.optimizers.SGD(),  
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']        #keras.metrics.sparse_categorical_accuracy
)

history = model.fit(
    x_train,
    y_train,
    batch_size=None,
    epochs=10,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    #validation_data=(x_test, y_test),
)

NameError: ignored